In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split

2023-10-09 14:10:18.619667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 14:10:19.187657: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-09 14:10:19.187707: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-09 14:10:19.187711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 2

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import datasets

train = pd.read_csv("trainsplitfinal.csv")
test = pd.read_csv("testsplitfinal.csv")
val = pd.read_csv("valsplitfinal.csv")

In [4]:
#shuffle the datasets before splitting into X and Y
train = train.sample(frac=1, random_state=42)
test = test.sample(frac=1, random_state=42)
val = val.sample(frac=1, random_state=42)

In [5]:
train.groupby(["labels"]).size()

labels
L_Alanine     92
L_Arginine    99
dtype: int64

In [6]:
test.groupby(["labels"]).size()

labels
L_Alanine     20
L_Arginine    21
dtype: int64

In [7]:
val.groupby(["labels"]).size()

labels
L_Alanine     20
L_Arginine    22
dtype: int64

In [8]:
#train
#one hot encode labels
df_dummies_train = pd.get_dummies(train['labels'])
df_class_neural_train = pd.concat([train,df_dummies_train],axis=1)
df_class_neural_train.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,labels,L_Alanine,L_Arginine
152,0.489714,0.069321,-0.513313,-0.315420,-0.700096,-0.271102,0.349693,0.465177,1.208912,1.910212,...,-0.495988,1.195777,1.058439,2.030700,1.427352,1.488084,0.489714,L_Alanine,1,0
75,0.489967,0.153725,0.438141,-0.198934,-0.359919,-0.411785,0.258048,0.494789,0.471588,1.171430,...,-0.385197,-0.455642,-0.241196,-0.188057,0.279314,0.539710,0.489967,L_Arginine,0,1
158,0.533873,0.517810,0.199092,0.290575,0.406079,0.478607,0.656255,0.879306,1.108907,1.199632,...,-0.647327,-0.055080,0.002162,0.471576,0.647556,0.526402,0.533873,L_Arginine,0,1
66,0.213644,-0.019065,-0.022161,-0.019917,-0.789178,0.324440,0.003031,0.533225,1.141834,1.401378,...,0.101493,0.480590,0.687032,0.621243,0.495430,0.236714,0.213644,L_Alanine,1,0
60,0.525893,0.233963,0.685102,0.555405,-0.234053,0.278221,0.466287,0.167396,0.889858,1.199905,...,0.522623,0.731315,0.867526,0.791746,0.811133,0.702320,0.525893,L_Arginine,0,1


In [9]:
#test
#one hot encode labels
df_dummies_test = pd.get_dummies(test['labels'])
df_class_neural_test = pd.concat([test,df_dummies_test],axis=1)
df_class_neural_test.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,labels,L_Alanine,L_Arginine
24,1.627433,1.129668,0.399811,0.329372,-0.345727,-0.869556,-0.198755,0.536544,-0.347216,-0.405334,...,1.258546,2.208351,2.179613,1.614505,2.514132,1.516275,1.627433,L_Arginine,0,1
13,0.114051,0.031418,-0.264262,-0.886127,-0.558256,-0.778959,-0.759516,-0.228709,-0.222577,0.325565,...,0.716719,0.397274,0.182710,0.028726,0.199658,0.043318,0.114051,L_Arginine,0,1
8,0.844626,0.745589,0.867735,0.866199,1.151742,0.165017,0.332068,1.037330,0.708769,0.476439,...,2.405155,1.391018,1.352265,2.247645,1.005213,2.071064,0.844626,L_Arginine,0,1
25,0.692678,0.624679,0.897541,0.487217,0.143851,-0.045510,0.436702,0.622413,1.056184,1.899240,...,0.428470,0.901580,1.043873,0.950765,0.709510,0.547874,0.692678,L_Alanine,1,0
4,-0.432895,-0.006113,0.316363,0.546559,0.761573,0.743488,0.662611,0.830290,1.210904,1.576646,...,-0.296227,-0.265973,-0.144571,-0.238312,-0.310430,-0.329081,-0.432895,L_Arginine,0,1


In [10]:
#val
#one hot encode labels
df_dummies_val = pd.get_dummies(val['labels'])
df_class_neural_val = pd.concat([val,df_dummies_val],axis=1)
df_class_neural_val.head()

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,labels,L_Alanine,L_Arginine
25,1.210246,0.119403,-0.102690,-0.601857,-0.425838,-0.267906,0.036628,0.651098,1.207193,0.948348,...,1.230415,1.495991,1.531078,1.651481,1.636371,1.672060,1.210246,L_Alanine,1,0
13,0.964164,1.019786,0.901728,-0.016188,-0.089121,0.714686,0.636101,-0.204933,0.374777,0.990461,...,-0.400608,0.883419,1.096213,1.385262,1.425094,0.947387,0.964164,L_Arginine,0,1
8,1.119232,0.839111,0.973988,0.827992,0.501022,0.534239,-0.298303,0.318920,0.983388,0.920092,...,1.498877,1.574368,1.638158,1.443904,1.057813,0.870994,1.119232,L_Alanine,1,0
26,0.883128,1.017666,0.653553,0.351255,0.313443,0.689685,0.338849,0.039009,0.038261,0.295629,...,0.567848,0.946857,1.421321,1.736713,1.419006,0.991768,0.883128,L_Arginine,0,1
4,1.662353,1.656257,1.465377,1.379148,1.524141,1.343944,1.215897,1.263876,1.233929,1.085183,...,0.448602,0.951779,1.009880,1.611717,1.641204,2.389964,1.662353,L_Arginine,0,1


In [11]:
X_train = df_class_neural_train.iloc[:,:72]
y_train = df_class_neural_train.iloc[:,73:]
X_test = df_class_neural_test.iloc[:,:72]
y_test = df_class_neural_test.iloc[:,73:]
X_val = df_class_neural_val.iloc[:,:72]
y_val = df_class_neural_val.iloc[:,73:]

In [12]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(191, 72)
(191, 2)
(41, 72)
(41, 2)
(42, 72)
(42, 2)


In [13]:
#save the splits
X_train.to_csv("finalXtrain.csv",index=False)
y_train.to_csv("finalYtrain.csv",index=False)
X_test.to_csv("finalXtest.csv",index=False)
y_test.to_csv("finalYtest.csv",index=False)
X_val.to_csv("finalXval.csv",index=False)
y_val.to_csv("finalYval.csv",index=False)